# 3.3 线性回归的简洁实现
- **目录**
  - 3.3.1 生成数据集
  - 3.3.2 读取数据集
  - 3.3.3 定义模型
  - 3.3.4 初始化模型参数
  - 3.3.5 定义损失函数
  - 3.3.6 定义优化算法
  - 3.3.7 训练

- 上一节的线性回归从头开始实现使用了Pytorch如下技术：
  - （1）通过张量来进行数据存储和线性代数；
  - （2）通过自动微分来计算梯度。
- 实际上，Pytorch等深度学习框架也实现了**数据迭代器**、**损失函数**、**优化器**和**神经网络层**等组件，可以直接使用，本节使用这些组件简洁实现线性回归。

## 3.3.1 生成数据集
- 首先**生成数据集**。

In [15]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [17]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
# 此处调用d2l包的synthetic_data函数，此函数与上一节的同名函数功能相同
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 3.3.2 读取数据集
- **调用框架中现有的API来读取数据**：
  - 将`features`和`labels`作为API的参数传递。
  -  通过数据迭代器指定`batch_size`。
  - 布尔值`is_train`表示是否希望数据迭代器对象在每个迭代周期内**打乱**数据。


In [18]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""    
    '''
    注意*号的用法，此处是对集合对象解包unpacking。
    data_arrays传递进来的是(features, labels),该数据是一个元组，
    但是TensorDataset的参数是一个个张量，因此需要将(features, labels)解包成
    features和labels两个张量，然后传递给TensorDataset。
    官方文档："data.TensorDataset(*tensors: torch.Tensor) -> None"，参数*tensor表示传入
    不确定个数的张量作为参数。
    
    '''
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

------------
- <b>说明：`data.TensorDataset(*data_arrays)`中的*操作符</b>
  - 这是Python中的解包（unpacking）操作符，也被称为星号运算符(star operator)。
  - 一般来说，星号运算符有两种用法：
    - 1. **在函数定义中**：用于接收不确定数量的参数，生成一个元组或者列表；
    - 2. **在函数调用或其他操作中**：用于解开序列类型或者映射类型的数据，进行元素级别的操作。
  - 在上述代码中，`*data_arrays` 就是用星号运算符将 `data_arrays` 这个元组拆分开。
  - 对于这样一个调用 `data.TensorDataset(*data_arrays)` ，如果不使用星号，那么 `TensorDataset` 会把 `data_arrays` 整体作为一个参数。
  - 使用星号后，`data_arrays` 中的每个元素（在这里是 `features` 和 `labels`两个tensor）都分别作为一个参数传入 `TensorDataset`。
  - 简单来说，在此场景下，`*`操作符起到了解包的作用，将 `data_arrays` 元组中的元素拆解开来，分别作为TensorDataset的参数。

------------

In [19]:
batch_size = 10

## 注意第一个参数是一个元组，在调用函数时需要解包
data_iter = load_array((features, labels), batch_size)

- 使用`iter`构造Python迭代器，并使用`next`从迭代器中获取第一项。


In [20]:
# 在进行迭代时，返回的是两个张量分别代表一个feature，一个label
next(iter(data_iter))

[tensor([[ 1.7421, -0.2556],
         [ 0.1006,  1.4299],
         [ 1.2421,  1.1366],
         [ 0.7422,  1.2734],
         [-0.0406, -1.4079],
         [-0.7890, -1.8570],
         [ 1.4191,  1.2250],
         [ 0.1268, -2.1254],
         [ 0.5341,  0.7830],
         [-0.4631, -0.4672]]),
 tensor([[ 8.5505],
         [-0.4506],
         [ 2.8317],
         [ 1.3722],
         [ 8.8949],
         [ 8.9314],
         [ 2.8669],
         [11.6952],
         [ 2.6064],
         [ 4.8867]])]

## 3.3.3 定义模型
- **使用pytorch框架预定义层优势：**
  - 只需关注使用哪些层来构造模型，而不必关注层的实现细节。
  - 开始定义一个模型变量`net`，它是一个`Sequential`类的实例。
  - `Sequential`类将多个层**串联**在一起。
  - 当给定输入数据时，`Sequential`实例将数据传入到第一层，然后将第一层的输出作为第二层的输入，以此类推。
  - 本例的模型只包含一个层，这一单层被称为**全连接层（fully-connected layer）**，其每一个输入都通过矩阵-向量乘法得到它的每个输出。
  - 由于本例是单层，实际上不需要`Sequential`，这里使用`Sequential`是符合**标准的流水线**，为后面的示例代码座位。

- 在PyTorch中，全连接层在`Linear`类中定义。(Tensorflow在Dense中定义)。
- 值得注意的是，将两个参数传递到`nn.Linear`中：
  - 第一个指定输入特征形状，即2。
  - 第二个指定输出特征形状，输出特征形状为单个标量，因此为1。


In [21]:
# nn是神经网络的缩写
from torch import nn
# Sequential类生成标准流水线，Linear是全连接层
## 参数里的2表示输入数据的列数，1表示输出数据的列数
net = nn.Sequential(nn.Linear(2, 1))

## 3.3.4 初始化模型参数

- 在使用`net`之前，需<b>初始化模型参数</b>，线性回归模型的参数包括权重和偏置。
- 深度学习框架通常有**预定义方法**来初始化参数。
- 可**直接访问参数**以设定它们的初始值：
  - 通过`net[0]`选择网络中的第一个图层，
  - 使用`weight.data`和`bias.data`方法访问参数。
  - 使用替换方法`normal_`和`fill_`来重写参数值。
  - 此处指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样；偏置参数将初始化为0。

In [22]:
## 注意Python函数以下划线结尾表示数据是就地(in-place)操作
net[0].weight.data.normal_(0, 0.01) #Linear中的weights与bias都有自带的初始化函数
net[0].bias.data.fill_(0) #很方便

tensor([0.])

In [23]:
# net的类型以及构成层
net,type(net),type(net[0]), type(net[0].bias), type(net[0].weight.data)

(Sequential(
   (0): Linear(in_features=2, out_features=1, bias=True)
 ),
 torch.nn.modules.container.Sequential,
 torch.nn.modules.linear.Linear,
 torch.nn.parameter.Parameter,
 torch.Tensor)

In [9]:
# torch的Tensor类封装所有与张量计算相关的函数
#dir(torch.Tensor)

## 3.3.5 定义损失函数


- **计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数**。
  - 默认情况下，它返回所有样本损失的平均值。
  - `nn.MSELoss(size_average=None, reduce=None, reduction: str = 'mean')`前两个参数以及废弃，`reduction`参数将作为唯一参数保存，默认值是`mean`，即求梯度的均值。本例中即小批量的梯度均值。


In [24]:
# 深度学习相关的类与函数几乎都在nn这个包中
loss = nn.MSELoss()

In [73]:
# MSELoss的官方文档
# nn.MSELoss?

## 3.3.6 定义优化算法


- 定义优化算法：
  - 小批量随机梯度下降算法是一种优化神经网络的标准工具，PyTorch在`optim`模块中实现了该算法的许多变种，后面有详细介绍。
  - **实例化一个`SGD`实例**时，需指定待优化的参数，可通过`net.parameters()`从模型中获得。
  - 还需指定优化算法所需的超参数字典，小批量随机梯度下降只需要设置`lr`值，这里设置为0.03。


In [25]:
# 在此处已经将net的参数指定给优化器了
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [26]:
# 参数的初始化值
# 包括weights与bias
net.parameters(), list(net.parameters())

(<generator object Module.parameters at 0x000001BAF13673E0>,
 [Parameter containing:
  tensor([[-0.0011,  0.0007]], requires_grad=True),
  Parameter containing:
  tensor([0.], requires_grad=True)])

In [11]:
# SGD官方文档
# torch.optim.SGD?

## 3.3.7 训练
- 通过深度学习框架的高级API来实现模型时，只需要相对较少的代码。
  - **不必单独分配参数、不必定义我们的损失函数，也不必手动实现小批量随机梯度下降**。
  - 当设计与开发复杂模型时，高级API的优势将大大增加。
- 开始训练时，在每个迭代周期里，将**完整遍历一次数据集（`train_data`）**，不停地从中获取一个小批量的输入和相应的标签。
- 对于每一个小批量进行以下训练步骤:
  * 调用`net(X)`生成预测并计算损失`l`，即**前向传播**。
  * 通过`l.backward()`进行**反向传播**来计算梯度。
  * 通过调用优化器trainer来**更新模型参数**。
- 为了更好的衡量训练效果，我们计算每个迭代周期后的损失，并打印它来监控训练过程。


In [27]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad() #将梯度归零，避免累加
        l.backward() #这一步只是计算出了梯度
        
        #trainer名为训练器，其实质就是一个优化器，用于更新参数
        #注意是通过step函数来更新参数的
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000229
epoch 2, loss 0.000103
epoch 3, loss 0.000103


- **比较生成数据集的真实参数和通过有限数据训练获得的模型参数**。
  - 要访问参数，首先从`net`访问所需的层，然后读取该层的权重和偏置。
  - 与从零开始实现中一样，简洁实现估计得到的参数与生成数据的真实参数非常接近。


In [28]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0002, 0.0001])
b的估计误差： tensor([5.6267e-05])


In [29]:
w,b,true_w,true_b

(tensor([[ 1.9998, -3.4001]]),
 tensor([4.1999]),
 tensor([ 2.0000, -3.4000]),
 4.2)

---------------------

- **说明：Python的星号*在Python中修饰变量时的作用**
- **（1）收集参数（在函数定义中）：**
  - *args：当定义函数时，在参数前加一个星号表示将所有位置参数收集到一个名为args的元组中。这允许函数接受任意数量的位置参数。
  - **kwargs：双星号用于收集关键字参数到一个名为kwargs的字典中。

In [30]:
def func(*args, **kwargs):
    print(args)   # args是一个包含所有未命名位置参数的元组
    print(kwargs)  # kwargs是一个包含所有命名关键字参数的字典

- **（2）参数解包（在函数调用或其他场景中）：**
  - *iterable：在调用函数时，一个星号用来将可迭代对象（如列表、元组）解包(unpacking)成位置参数。
  - **dict：在调用函数时，两个星号用来将字典对象解包成关键字参数。

In [31]:
args = (1, 2, 3)
kwargs = {'a': 4, 'b': 5}
func(*args, **kwargs)  # 调用func函数，位置参数为1, 2, 3，关键字参数为a=4, b=5

(1, 2, 3)
{'a': 4, 'b': 5}


- **（3）在列表、元组和字符串等可迭代对象的上下文中：**
  - *iterable：星号用于列表、元组等可迭代对象，可以在创建新列表或元组时将一个可迭代对象的元素解包成独立的元素。
  - 参见下例：

In [32]:
numbers = [1, 2, 3]
more_numbers = [*numbers, 4, 5]  # 创建新列表，开始时包含numbers中的元素，后跟4和5
print(more_numbers)  # 输出：[1, 2, 3, 4, 5]

# 在元组中使用
values = (1, 2, 3)
more_values = *values, 4
print(more_values)  # 输出：(1, 2, 3, 4)

[1, 2, 3, 4, 5]
(1, 2, 3, 4)


- **（4）在变量赋值中：**
  - *variable：当在变量赋值时使用星号，比如在多变量赋值中，可以用来收集多余的值到一个列表中。

In [33]:
a, *b, c = [1, 2, 3, 4, 5]
print(a)  # 输出：1
print(b)  # 输出：[2, 3, 4]
print(c)  # 输出：5

1
[2, 3, 4]
5


--------

## 小结


* 可以使用PyTorch的高级API更简洁地实现模型。
* 在PyTorch中，`data`模块提供了数据处理工具，`nn`模块定义了大量的神经网络层和常见损失函数。
* 可以通过`_`结尾的方法将参数替换，从而初始化参数。
